In [1]:
%run common.ipynb

[tokenizer.decode(i) for i in tokenizer.get_batch_data(prefix=True)[1]][:10]

['S字母:5278=wqqqrqqqrqqqrqqqwE',
 'S大写:9113=叁陆肆伍伍陆肆伍伍陆肆伍伍陆肆伍贰E',
 'S小写:1255=五〇二〇五〇二〇五〇二〇五〇二〇EP',
 'S大写:1198=肆柒玖贰肆柒玖贰肆柒玖贰肆柒玖贰EP',
 'S大写:3751=壹伍零零伍伍零零伍伍零零伍伍零零肆E',
 'S大写:5649=贰贰伍玖捌贰伍玖捌贰伍玖捌贰伍玖陆E',
 'S小写:6892=二七五七〇七五七〇七五七〇七五六八E',
 'S小写:2195=八七八〇八七八〇八七八〇八七八〇EP',
 'S字母:6627=wytqpytqpytqpytpiE',
 'S大写:7516=叁零零陆柒零零陆柒零零陆柒零零陆肆E']

In [2]:
@torch.no_grad()
def get_question_and_answer():
    _, token, _ = tokenizer.get_batch_data(prefix=True)

    split = [i.index(tokenizer.encoder['=']) + 1 for i in token]

    #只要问题部分,等号后面的内容切除
    question = [t[:s] for t, s in zip(token, split)]
    answer = [t[s:] for t, s in zip(token, split)]

    #统一长度
    lens = max([len(i) for i in question])
    question = [[tokenizer.encoder['P']] * (lens - len(i)) + i
                for i in question]
    question = torch.LongTensor(question).to(device)

    lens = max([len(i) for i in answer])
    answer = [[tokenizer.encoder['P']] * (lens - len(i)) + i for i in answer]
    answer = torch.LongTensor(answer).to(device)

    return question, answer


question, answer = get_question_and_answer()

question.shape, answer.shape

(torch.Size([64, 9]), torch.Size([64, 18]))

In [3]:
model_ppo = torch.load('ppo.model')
model_ppo.to(device)
model_ppo.eval()

predict = generate(model_ppo.model_gen, question)
predict = predict[:, question.shape[1]:]

predict.shape

torch.Size([64, 18])

In [4]:
correct = 0
for q, a, p in zip(question, answer, predict):
    q, a, p = q.tolist(), a.tolist(), p.tolist()

    if tokenizer.encoder['E'] in a:
        split = a.index(tokenizer.encoder['E']) + 1
        a = a[:split]

    if tokenizer.encoder['E'] in p:
        split = p.index(tokenizer.encoder['E']) + 1
        p = p[:split]

    q, a, p = tokenizer.decode(q), tokenizer.decode(a), tokenizer.decode(p)

    print(q, a, p)

    correct += a == p

correct / len(answer)

S大写:7664= 叁零陆伍玖零陆伍玖零陆伍玖零陆伍陆E 叁零陆伍玖零陆伍玖零陆伍玖零陆伍陆E
S小写:9893= 三九五七五九五七五九五七五九五七二E 三九五七五九五七五九五七五九五七二E
S小写:6529= 二六一一八六一一八六一一八六一一六E 二六一一八六一一八六一一八六一一六E
S小写:8139= 三二五五九二五五九二五五九二五五六E 三二五五九二五五九二五五九二五五六E
S大写:7086= 贰捌叁肆陆捌叁肆陆捌叁肆陆捌叁肆肆E 贰捌叁肆陆捌叁肆陆捌叁肆陆捌叁肆肆E
S大写:5099= 贰零叁玖捌零叁玖捌零叁玖捌零叁玖陆E 贰零叁玖捌零叁玖捌零叁玖捌零叁玖陆E
S小写:9199= 三六七九九六七九九六七九九六七九六E 三六七九九六七九九六七九九六七九六E
S大写:6229= 贰肆玖壹捌肆玖壹捌肆玖壹捌肆玖壹陆E 贰肆玖壹捌肆玖壹捌肆玖壹捌肆玖壹陆E
S小写:6546= 二六一八六六一八六六一八六六一八四E 二六一八六六一八六六一八六六一八四E
S数字:4435= 17741774177417740E 17741774177417740E
S字母:3113= qwrtewrtewrtewrtwE qwrtewrtewrtewrtwE
S小写:1271= 五〇八四五〇八四五〇八四五〇八四E 四〇八四四〇八四四〇八四四〇八四E
S小写:1424= 五六九六五六九六五六九六五六九六E 五六九六五六九六五六九六五六九六E
S大写:4742= 壹捌玖陆玖捌玖陆玖捌玖陆玖捌玖陆捌E 壹捌玖陆玖捌玖陆玖捌玖陆玖捌玖陆捌E
S数字:3692= 14769476947694768E 14769476947694768E
S数字:7171= 28686868686868684E 28686868686868684E
S大写:2777= 壹壹壹零玖壹壹零玖壹壹零玖壹壹零捌E 壹壹壹零玖壹壹零玖壹壹零玖壹壹零捌E
S小写:9911= 三九六四七九六四七九六四七九六四四E 三九六四七九六四七九六四七九六四四E
S数字:4613= 18453845384538452E 18453845384538452E
S大写:2593= 壹零叁柒叁零叁柒叁零叁柒叁零叁柒贰E 壹零叁柒叁零叁柒叁零叁柒叁零叁柒贰E
S小写:9875= 三九五〇三九五〇三九五〇三九五〇〇E 三九五〇三九五〇三九五〇三九五

0.984375